In [3]:
import random
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

## 데이터 불러오기

In [137]:
menu = pd.read_csv('../../Capston/data/menu.csv', encoding='utf-8')
menu.head()

,id,category,name,price,emotion,weather,image,restaurant_id
0,1,국밥,양평해장국,9000,"배고픔,기쁨,피곤,스트레스","맑음,구름,비,눈",menu/Restaurant object (1)/양평해장국.jpg,1
1,2,국밥,양평해장국(특),10000,"배고픔,기쁨,피곤,스트레스","맑음,구름,비,눈",menu/Restaurant object (1)/양평해장국특.jpg,1
2,3,국밥,뼈다귀해장국,10000,"배고픔,기쁨,피곤,스트레스","맑음,구름,비,눈",menu/Restaurant object (1)/뼈다귀해장국.jpg,1
3,4,국밥,얼큰순대국,9000,"배고픔,기쁨,피곤,스트레스","맑음,구름,비,눈",menu/Restaurant object (1)/얼큰순대국.jpg,1
4,5,국밥,내장탕,13000,"배고픔,기쁨,피곤,스트레스","맑음,구름,비,눈",menu/Restaurant object (1)/내장탕.jpg,1


In [138]:
menu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323 entries, 0 to 322
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             323 non-null    int64 
 1   category       323 non-null    object
 2   name           323 non-null    object
 3   price          323 non-null    int64 
 4   emotion        323 non-null    object
 5   weather        323 non-null    object
 6   image          251 non-null    object
 7   restaurant_id  323 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 20.3+ KB


In [112]:
nutrient_menu = pd.read_csv('../../Capston/data/nutrient.csv')
nutrient_menu.head()

,id,name,gram,calorie,carbohydrate,protein,fat,saturatedfat,unsaturatedfat,cholesterol,sodium,potash,ingredient,allergy,menu_id
0,1,뼈해장국,300.0,315.0,7.10,39.940,13.540,4.480,6.0900,106.0,2420.0,626.0,"돼지고기, 우거지, 대파, 마늘",고기,3
1,2,양평해장국,500.0,662.0,16.47,83.210,28.560,9.360,12.7370,222.0,4199.0,1363.0,"마늘, 생강, 대파",NaN,1
2,3,양평해장국(특),600.0,794.4,197.64,99.852,34.272,11.232,15.2844,266.0,5039.0,1636.0,"마늘, 생강, 대파",NaN,2
3,4,얼큰순대국,300.0,328.0,33.78,23.250,10.350,3.615,3.1450,439.0,1953.0,485.0,"순대, 사골, 마늘, 대파, 쌀, 돼지고기",고기,4
4,5,육개장,200.0,165.0,10.45,14.320,7.570,2.083,3.2890,20.0,720.0,429.0,"소고기, 고사리, 대파, 숙주",NaN,6


In [113]:
nutrient_menu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323 entries, 0 to 322
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              323 non-null    int64  
 1   name            323 non-null    object 
 2   gram            323 non-null    float64
 3   calorie         323 non-null    float64
 4   carbohydrate    260 non-null    float64
 5   protein         323 non-null    float64
 6   fat             260 non-null    float64
 7   saturatedfat    287 non-null    float64
 8   unsaturatedfat  136 non-null    float64
 9   cholesterol     188 non-null    float64
 10  sodium          317 non-null    float64
 11  potash          149 non-null    float64
 12  ingredient      323 non-null    object 
 13  allergy         287 non-null    object 
 14  menu_id         323 non-null    int64  
dtypes: float64(10), int64(2), object(3)
memory usage: 38.0+ KB


In [13]:
restaurant = pd.read_csv('../../Capston/data/restaurant.csv')
restaurant.head()

,id,name,address,business_hours,phone_number,category_name,image,longitude,latitude
0,1,일품양평해장국 인천대점,인천 연수구 송도동 13-58,10:00 ~ 22:00,032-214-0359,음식점 > 한식 > 해장국 > 일품양평해장국,restaurant/일품양평해장국로고.jpg,126.636,37.3740
1,2,스노우폭스 송도점,인천 연수구 송도동 13-58,07:30 ~ 20:00,032-710-6464,"음식점 > 일식 > 초밥,롤",restaurant/스노우폭스_로고.jpg,126.637,37.3740
2,3,오백국수 송도인천대점,인천 연수구 송도동 13-58,11:00 ~ 20:00,032-859-7788,음식점 > 한식 > 국수,restaurant/오백국수로고.jpg,126.637,37.3740
3,4,맘스터치 인천대제3기숙사점,인천 연수구 송도동 12-1,11:00 ~ 20:00,032-858-3211,음식점 > 패스트푸드 > 맘스터치,restaurant/맘스터치로고.jpg,126.630,37.3732
4,5,샐러디 인천대점,인천 연수구 송도동 12-1,10:00 ~ 20:00,070-4155-2222,음식점 > 샐러드 > 샐러디,restaurant/샐러디.jpg,126.632,37.3729


In [14]:
restaurant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              18 non-null     int64  
 1   name            18 non-null     object 
 2   address         18 non-null     object 
 3   business_hours  18 non-null     object 
 4   phone_number    8 non-null      object 
 5   category_name   18 non-null     object 
 6   image           7 non-null      object 
 7   longitude       18 non-null     float64
 8   latitude        18 non-null     float64
dtypes: float64(2), int64(1), object(6)
memory usage: 1.4+ KB


## 임의의 사용자 정보 작성

In [15]:
user = pd.DataFrame({'id' : [1, 2, 3, 4, 5],
                     'email' : ['abc', 'dfe', 'efg', 'hij', 'lmn'],
                     'username' : ['a', 'b', 'c', 'd', 'e'],
                     'nickname' : ['a', 'b', 'c', 'd', 'e'],
                     'gender' : ['male', 'female', 'male', 'male', 'female'],
                     'age' : [23, 24, 25, 26, 27]})
user

,id,email,username,nickname,gender,age
0,1,abc,a,a,male,23
1,2,dfe,b,b,female,24
2,3,efg,c,c,male,25
3,4,hij,d,d,male,26
4,5,lmn,e,e,female,27


In [16]:
user_allergy = pd.DataFrame({'email' : ['abc', 'dfe', 'efg', 'hij', 'lmn'],
                             '달걀': [0, 0, 0, 0, 0],
                             '우유': [1, 1, 0, 0 ,0],
                             '밀': [0, 0, 0, 0, 1],
                             '콩': [0, 0, 1, 0, 0], 
                             '땅콩': [0, 0, 0, 0, 1], 
                             '생선': [1, 1, 0, 0, 0], 
                             '고기': [0, 0, 0, 0, 0], 
                             '조개': [0, 0, 0, 0, 1],
                             '갑각류': [1, 0, 0, 0, 0]})
user_allergy

,email,달걀,우유,밀,콩,땅콩,생선,고기,조개,갑각류
0,abc,0,1,0,0,0,1,0,0,1
1,dfe,0,1,0,0,0,1,0,0,0
2,efg,0,0,0,1,0,0,0,0,0
3,hij,0,0,0,0,0,0,0,0,0
4,lmn,0,0,1,0,1,0,0,1,0


## 기능별 함수 작성

### [기능 1] 사용자 정보에서 알러지 정보 제외하고 추천

In [17]:
allergy_list = list(user_allergy)
print(allergy_list)

['email', '달걀', '우유', '밀', '콩', '땅콩', '생선', '고기', '조개', '갑각류']


In [18]:
# 유저 이메일 받아오기 (임의로 지정)
user_email = 'efg'

user_index = ''
for i in range(len(user)):
    if user.iloc[i][1] == user_email:
        user_index = i
user_index

2

In [19]:
user_num = len(user)
allergy_num = len(allergy_list)

# 개인별 알러지 담아둘 리스트
personal_allergy = []

# 유저 정보에서 알러지에 해당하는 음식 리스트 불러오기
for i in range(allergy_num):
    
    if user_allergy.iloc[user_index][i] == 1:
            personal_allergy.append(allergy_list[i])
            
personal_allergy

['콩']

In [20]:
# nutrient 파일에서 해당 알러지가 없는 메뉴
not_allergy_index = []

for i in range(len(nutrient_menu)):
    allergy = str( nutrient_menu.iloc[i]['allergy'])
    allergy_list = allergy.split(',')
    
    # 해당 알러지가 있는 메뉴 제외한 메뉴 인덱스 리스트
    add_menu = ''
    
    for j in allergy_list:
        if j not in personal_allergy:
            add_menu = 1
    
    if add_menu == 1:
        not_allergy_index.append(i)

In [21]:
# 알러지 없는 메뉴 중 한 가지 랜덤 선택
choice = random.randrange(0, len(not_allergy_index))
choice

235

In [22]:
# 메뉴 추출
menu.iloc[not_allergy_index[choice]]

id                                                   249
category                                             아시안
name                                             매콤 물쌀국수
price                                               6900
emotion                                       적당,피곤,스트레스
weather                                        맑음,구름,비,눈
image            menu/Restaurant object (12)/매콤_물쌀국수.jpg
restaurant_id                                         12
Name: 241, dtype: object

### [기능 2] 사용자가 선택한 예산

In [25]:
# 예산 입력받기
want_price = int(input("얼마만큼의 예산을 원하십니까? (입력값 이하) "))

얼마만큼의 예산을 원하십니까? (입력값 이하) 5000


In [26]:
# 메뉴 리스트에서 해당 예산보다 작은 금액의 메뉴 리스트업
menu_num = len(menu['id'])
price_menu_index = []

for i in range(menu_num):
    if menu.iloc[i]['price'] <= want_price:
        price_menu_index.append(i)

print(price_menu_index)

[25, 26, 27, 28, 29, 44, 45, 46, 47, 48, 55, 56, 57, 58, 63, 66, 69, 70, 71, 72, 100, 101, 102, 103, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 151, 152, 154, 155, 156, 157, 167, 182, 183, 184, 192, 193, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 229, 234, 235, 246, 248, 249, 255, 256, 257, 258, 259, 265, 266, 267, 268, 269, 270, 271, 280, 288, 306, 307]


In [27]:
# 원하는 금액에 맞는 메뉴 랜덤 추천
choice = random.randrange(0, len(price_menu_index))
choice

60

In [28]:
# 메뉴 추출
menu.iloc[price_menu_index[choice]]

id                                                   243
category                                              양식
name                                            치즈오븐스파게티
price                                               5000
emotion                                      배고픔,기쁨,스트레스
weather                                               맑음
image            menu/Restaurant object (9)/치즈오븐스파게티.jpg
restaurant_id                                          9
Name: 235, dtype: object

### [기능 3] 사용자가 선택한 날씨, 기분

* 기분 (emotion) : 배고픔, 적당, 기쁨, 피곤, 스트레스
* 날씨 (weather) : 맑음, 구름, 비, 눈, 황사

In [29]:
# 입력받은 날씨
user_weather = input('오늘의 날씨를 입력해주세요 (맑음, 구름, 비, 눈, 황사 중 하나) : ')

오늘의 날씨를 입력해주세요 (맑음, 구름, 비, 눈, 황사 중 하나) : 눈


In [30]:
# 메뉴 리스트에서 해당 날씨에 해당하는 메뉴 리스트업
menu_weather_list = []

for i in range(menu_num):
    weather_list = menu.iloc[i]['weather'].split(',')
    
    add_menu = ""
    for j in weather_list:
        if j == user_weather:
            add_menu = 1
    
    if add_menu == 1:
        menu_weather_list.append(i)

len(menu_weather_list)

49

In [31]:
# 날씨에 맞는 메뉴 랜덤 추천
choice = random.randrange(0, len(menu_weather_list))
choice

11

In [32]:
# 메뉴 추출
menu.iloc[menu_weather_list[choice]]

id                                                  41
category                                            한식
name                                            매생이칼국수
price                                             8000
emotion                                    배고픔,피곤,스트레스
weather                                      맑음,구름,비,눈
image            menu/Restaurant object (3)/매생이칼국수.jpg
restaurant_id                                        3
Name: 37, dtype: object

In [33]:
# 입력받은 기분
user_emotion = input('오늘의 기분을 입력해주세요 (배고픔, 적당, 기쁨, 피곤, 스트레스 중 하나) : ')

오늘의 기분을 입력해주세요 (배고픔, 적당, 기쁨, 피곤, 스트레스 중 하나) : 기쁨


In [34]:
# 메뉴 리스트에서 해당 기분에 해당하는 메뉴 리스트업
menu_emotion_list = []

for i in range(menu_num):
    emotion_list = menu.iloc[i]['emotion'].split(',')
    
    add_menu = ""
    for j in emotion_list:
        if j == user_emotion:
            add_menu = 1
    
    if add_menu == 1:
        menu_emotion_list.append(i)

len(menu_emotion_list)

114

In [35]:
# 날씨에 맞는 메뉴 랜덤 추천
choice = random.randrange(0, len(menu_emotion_list))
choice

8

In [36]:
# 메뉴 추출
menu.iloc[menu_emotion_list[choice]]

id                                               9
category                                        고기
name                                            편육
price                                         8000
emotion                                      적당,기쁨
weather                                         맑음
image            menu/Restaurant object (1)/편육.jpg
restaurant_id                                    1
Name: 8, dtype: object

In [37]:
# 날씨 + 기분 합친 버전
weather_emotion_list = []

for i in menu_weather_list:
    for j in menu_emotion_list:
        if i == j:
            weather_emotion_list.append(i)

len(weather_emotion_list)

6

In [38]:
choice = random.randrange(0, len(weather_emotion_list))
menu.iloc[weather_emotion_list[choice]]

id                                                  4
category                                           국밥
name                                            얼큰순대국
price                                            9000
emotion                                배고픔,기쁨,피곤,스트레스
weather                                     맑음,구름,비,눈
image            menu/Restaurant object (1)/얼큰순대국.jpg
restaurant_id                                       1
Name: 3, dtype: object

### [기능 4] 사용자가 위시리스트에 담은 메뉴를 비교하여 추천

In [39]:
# PreferredMenu에 담긴 메뉴 확인 (임의로 만듦)
preferred_menu = pd.DataFrame({'preference' : ['abc', 'dfe', 'efg', 'hij', 'lmn'],
                               'user_id' : ['1', '2', '3', '4', '1'],
                               'menu_id' : ['52', '42', '31', '42', '53']})
preferred_menu

,preference,user_id,menu_id
0,abc,1,52
1,dfe,2,42
2,efg,3,31
3,hij,4,42
4,lmn,1,53


In [40]:
# 유저 아이디 받기 (임의로 설정)
user_id = '1'

# 해당 유저가 선호하는 음식에 담은 메뉴 정보 불러오기
user_prefer_menu = []

for i in range(len(preferred_menu)):
    if preferred_menu.iloc[i]['user_id'] == '1':
        user_prefer_menu.append(preferred_menu.iloc[i]['menu_id'])

user_prefer_menu

['52', '53']

In [41]:
# 메뉴 아이디의 인덱스 가져오기
prefer_menu_index = []

for i in range(len(user_prefer_menu)):
    for j in range(len(menu)):
        if int(user_prefer_menu[i]) == menu.iloc[j]['id']:
            prefer_menu_index.append(j)
            
prefer_menu_index

[48, 49]

In [42]:
# 메뉴 아이디로 메뉴 정보 불러오기 (기본 메뉴정보, 영양정보 속성)
prefer_menu = menu.loc[prefer_menu_index]
prefer_menu

,id,category,name,price,emotion,weather,image,restaurant_id
48,52,한식,멸치주먹밥,2500,적당,맑음,menu/Restaurant object (3)/멸치주먹밥.jpg,3
49,53,한식,콩국수(여름메뉴),8000,"배고픔,피곤,스트레스","맑음,구름,비",menu/Restaurant object (3)/콩국수여름메뉴.jpg,3


### Feature

- 특징 데이터 만들기

In [135]:
# nutrient_menu 데이터의 ingredient 피처를 menu_id 기준으로 정렬
nutrient_menu = nutrient_menu.sort_values(by=['menu_id'])

In [147]:
tmp_df = nutrient_menu['ingredient']
tmp_df = tmp_df.reset_index(drop=True)
tmp_df

0                          마늘, 생강, 대파
1                          마늘, 생강, 대파
2                   돼지고기, 우거지, 대파, 마늘
3             순대, 사골, 마늘, 대파, 쌀, 돼지고기
4                       내장, 파, 된장, 마늘
                    ...              
318    새우, 양파, 달걀, 전분가루, 고추장, 마늘, 밀가루
319    새우, 양파, 달걀, 전분가루, 고추장, 마늘, 밀가루
320    새우, 양파, 달걀, 전분가루, 고추장, 마늘, 밀가루
321                 밀가루, 닭고기, 양상추, 치즈
322           밀가루, 돼지고기, 토마토, 달걀, 양상추
Name: ingredient, Length: 323, dtype: object

In [150]:
# 메뉴 데이터 특징 피처 만들기
menu['feature'] = menu['category'] + " " + menu['name'] + " " + menu['weather'] + " " + menu['emotion'] + " " + tmp_df
menu.tail()

,id,category,name,price,emotion,weather,image,restaurant_id,feature
318,343,중식,깐소 새우(소),32000,적당,"맑음,구름,황사",NaN,7,"중식 깐소 새우(소) 맑음,구름,황사 적당 새우, 양파, 달걀, 전분가루, 고추장,..."
319,344,중식,깐소 새우(중),48000,적당,"맑음,구름,황사",NaN,7,"중식 깐소 새우(중) 맑음,구름,황사 적당 새우, 양파, 달걀, 전분가루, 고추장,..."
320,345,중식,깐소 새우(대),64000,배고픔,"맑음,구름,황사",NaN,7,"중식 깐소 새우(대) 맑음,구름,황사 배고픔 새우, 양파, 달걀, 전분가루, 고추장..."
321,346,햄버거,딥치즈싸이버거,5100,"배고픔,피곤","맑음,구름",NaN,4,"햄버거 딥치즈싸이버거 맑음,구름 배고픔,피곤 밀가루, 닭고기, 양상추, 치즈"
322,347,햄버거,에그불고기버거,5200,"배고픔,피곤","맑음,구름",NaN,4,"햄버거 에그불고기버거 맑음,구름 배고픔,피곤 밀가루, 돼지고기, 토마토, 달걀, 양상추"


In [154]:
menu.feature.isnull().sum()

0

In [169]:
menu_feature = menu['feature'].tolist()

In [170]:
# TF-IDF Vectorizer Object 구현
tfidf = text.TfidfVectorizer(input=menu_feature)
tfidf_matrix = tfidf.fit_transform(menu_feature)
tfidf_matrix.shape

(323, 486)

In [172]:
# 코사인 유사도 계산
similarity = cosine_similarity(tfidf_matrix)

In [182]:
indices = pd.Series(menu.index, index=menu.feature).drop_duplicates()
indices

feature
국밥 양평해장국 맑음,구름,비,눈 배고픔,기쁨,피곤,스트레스 마늘, 생강, 대파                   0
국밥 양평해장국(특) 맑음,구름,비,눈 배고픔,기쁨,피곤,스트레스 마늘, 생강, 대파                1
국밥 뼈다귀해장국 맑음,구름,비,눈 배고픔,기쁨,피곤,스트레스 돼지고기, 우거지, 대파, 마늘           2
국밥 얼큰순대국 맑음,구름,비,눈 배고픔,기쁨,피곤,스트레스 순대, 사골, 마늘, 대파, 쌀, 돼지고기      3
국밥 내장탕 맑음,구름,비,눈 배고픔,기쁨,피곤,스트레스 내장, 파, 된장, 마늘                  4
                                                            ... 
중식 깐소 새우(소) 맑음,구름,황사 적당 새우, 양파, 달걀, 전분가루, 고추장, 마늘, 밀가루       318
중식 깐소 새우(중) 맑음,구름,황사 적당 새우, 양파, 달걀, 전분가루, 고추장, 마늘, 밀가루       319
중식 깐소 새우(대) 맑음,구름,황사 배고픔 새우, 양파, 달걀, 전분가루, 고추장, 마늘, 밀가루      320
햄버거 딥치즈싸이버거 맑음,구름 배고픔,피곤 밀가루, 닭고기, 양상추, 치즈                   321
햄버거 에그불고기버거 맑음,구름 배고픔,피곤 밀가루, 돼지고기, 토마토, 달걀, 양상추             322
Length: 323, dtype: int64

In [183]:
# 가장 유사한 메뉴 추천 함수
def rcm_prefer(feature, similarity=similarity):
    
    # 사용자가 좋아하는 메뉴 특성 계산
    index = indices[feature]
    
    # 유사도 계산
    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[0:10]
    restaurantindices = [i[0] for i in similarity_scores]
    
    return menu['name'].iloc[restaurantindices]

In [185]:
print(rcm_prefer("햄버거 에그불고기버거 맑음,구름 배고픔,피곤 밀가루, 돼지고기, 토마토, 달걀, 양상추"))

322       에그불고기버거
66          불고기버거
67       디럭스불고기버거
73     텍사스바베큐치킨버거
76      청양마요미트볼버거
68        새우불고기버거
57       양념치킨싸이버거
59        싸이플렉스버거
63           휠렛버거
72       할라피뇨통살버거
Name: name, dtype: object
